In [8]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import matplotlib.lines as mlines

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import scipy.spatial.distance as dist
np.random.seed(9999)

Считаем точность предсказанного

In [9]:
def accuracy(actual, predicted):
    return 1.0 - (actual != predicted).sum() / float(actual.shape[0])

# Это класс для описания функции ядра
По идее, их можно написать любых, длолжно работать с каждым

In [10]:
class RBF(object):
    def __init__(self, gamma=0.1):
        self.gamma = gamma

    def __call__(self, x, y):
        x = np.atleast_2d(x)
        y = np.atleast_2d(y)
        return np.exp(-self.gamma * dist.cdist(x, y) ** 2).flatten()

# Это класс для самописного метода главных векторов с rbf ядром 
Тут могу в краце описать как работает:

Остальное затехаю прямо в курсовой, оформлять формулы не было времени

In [31]:
class SVM():
    def __init__(self, C=1.0, kernel=None, tol=1e-3, max_iter=100):
       
        self.C = C
        self.tol = tol
        self.max_iter = max_iter
        if kernel is None:
            self.kernel = Linear()
        else:
            self.kernel = kernel

        self.b = 0
        self.alpha = None
        self.K = None
        self.train_loss = None

        
        
    def fit(self, X, y=None):
        self.X = X
        self.y = y
        train_loss_epoch = []
        if X.ndim == 1:
            self.n_samples, self.n_features = 1, X.shape
        else:
            self.n_samples, self.n_features = X.shape[0], np.prod(X.shape[1:])
        self.K = np.zeros((self.n_samples, self.n_samples))
        for i in range(self.n_samples):
            self.K[:, i] = self.kernel(self.X, self.X[i, :])
        self.alpha = np.zeros(self.n_samples)
        self.sv_idx = np.arange(0, self.n_samples)
        return self._train()

    
    
    def _train(self):
        iters = 0
        while iters < self.max_iter:
            iters += 1
            alpha_prev = np.copy(self.alpha)

            for j in range(self.n_samples):
                # Первый выбираем случайно
                i = self.random_index(j)

                eta = 2.0 * self.K[i, j] - self.K[i, i] - self.K[j, j]
                if eta >= 0:
                    continue
                L, H = self._find_bounds(i, j)

                # Находим ошибки
                e_i, e_j = self._error(i), self._error(j)

                # Сохраним пока старые alpha
                alpha_io, alpha_jo = self.alpha[i], self.alpha[j]

                # Найдем новые
                self.alpha[j] -= (self.y[j] * (e_i - e_j)) / eta
                self.alpha[j] = self.clip(self.alpha[j], H, L)

                self.alpha[i] = self.alpha[i] + self.y[i] * self.y[j] * (alpha_jo - self.alpha[j])

                 
                b1 = (
                    self.b - e_i - self.y[i] * (self.alpha[i] - alpha_io) * self.K[i, i]
                    - self.y[j] * (self.alpha[j] - alpha_jo) * self.K[i, j]
                )
                b2 = (
                    self.b - e_j - self.y[j] * (self.alpha[j] - alpha_jo) * self.K[j, j]
                    - self.y[i] * (self.alpha[i] - alpha_io) * self.K[i, j]
                )
                if 0 < self.alpha[i] < self.C:
                    self.b = b1
                elif 0 < self.alpha[j] < self.C:
                    self.b = b2
                else:
                    self.b = 0.5 * (b1 + b2)

            #Тут проверяем сходимость
            diff = np.linalg.norm(self.alpha - alpha_prev)
            if diff < self.tol:
                break

        # Нашли опорный вектор - сохранили индекс
        self.sv_idx = np.where(self.alpha > 0)[0]

        
        
        
        
    def _predict(self, X=None):
        n = X.shape[0]
        result = np.zeros(n)
        for i in range(n):
            result[i] = np.sign(self._predict_row(X[i, :]))
        return result

    
    
    def _predict_row(self, X):
        k_v = self.kernel(self.X[self.sv_idx], X)
        return np.dot((self.alpha[self.sv_idx] * self.y[self.sv_idx]).T, k_v.T) + self.b

    def clip(self, alpha, H, L):
        if alpha > H:
            alpha = H
        if alpha < L:
            alpha = L
        return alpha

    def _error(self, i):
        return self._predict_row(self.X[i]) - self.y[i]

    def _find_bounds(self, i, j):
        if self.y[i] != self.y[j]:
            L = max(0, self.alpha[j] - self.alpha[i])
            H = min(self.C, self.C - self.alpha[i] + self.alpha[j])
        else:
            L = max(0, self.alpha[i] + self.alpha[j] - self.C)
            H = min(self.C, self.alpha[i] + self.alpha[j])
        return L, H

    def random_index(self, z):
        i = z
        while i == z:
            i = np.random.randint(0, self.n_samples - 1)
        return i
    
    def predict(self, X=None):
        if not isinstance(X, np.ndarray):
            X = np.array(X)

        if self.X is not None or not self.fit_required:
            return self._predict(X)
        else:
            raise ValueError("Predict first, please")
            
   

# А тут грузим данные, делим на выборки, вызываем метод 
В результате выводим точность и смотрим, где разошлись значения и в какую сторону 

In [48]:
for i in range(4):
    X = pd.read_csv('moleculas/Matrices/cox2/cox2_matrix_alphabet_NNdbr_UpTo2chains.csv')
    X = X.to_numpy()
    
    y_real = pd.read_csv('moleculas/Matrices/cox2/cox2_y.csv')
    y_real = y_real.to_numpy()
    y_reals = y_real.flatten()
    
    y = pd.read_csv('moleculas/Matrices/cox2/cox2_y_bin.csv')
    y = y.to_numpy()
    y = y.flatten()
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2021)
    
    model = SVM(max_iter=500, kernel=RBF(gamma=0.1), C=1)
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print("Classification accuracy: %s" % (accuracy(y_test, predictions)))
    
    print ('y_test    predictions     real val.')
    for j in range(y_test.shape[0]):
        if(y_test[j] == predictions[j]):
            print('Equal.          ', y_real[j])
        else:
            print(y_test[j], '        ', predictions[j], '        ', y_real[j])

Classification accuracy: 0.6914893617021276
y_test    predictions     real val.
Equal.           ['[0.06, 0.06]']
-1          1.0          ['[0.51, 0.51]']
Equal.           ['[10.2, 10.2]']
-1          1.0          ['[0.06, 0.06]']
Equal.           ['[0.08, 0.08]']
Equal.           ['[0.04, 0.04]']
1          -1.0          ['[2.87, 2.87]']
1          -1.0          ['[0.25, 0.25]']
Equal.           ['(100.0, inf)']
Equal.           ['(100.0, inf)']
Equal.           ['(100.0, inf)']
Equal.           ['(100.0, inf)']
Equal.           ['(100.0, inf)']
-1          1.0          ['[0.014, 0.014]']
1          -1.0          ['(100.0, inf)']
-1          1.0          ['(100.0, inf)']
Equal.           ['(100.0, inf)']
Equal.           ['(100.0, inf)']
Equal.           ['[0.12, 0.12]']
Equal.           ['[1.61, 1.61]']
Equal.           ['[1.02, 1.02]']
Equal.           ['[0.06, 0.06]']
Equal.           ['[3.23, 3.23]']
-1          1.0          ['[0.75, 0.75]']
Equal.           ['[0.02, 0.02]']
Equa

# Сравним на том же разбиении со стандартным методом

In [49]:
from sklearn import svm as svm_orig
from sklearn.metrics import recall_score

In [50]:
C = 1.0 

model1 = svm_orig.SVC(kernel='rbf', gamma=0.1, C=C)


model1.fit(X_train, y_train)
y_predict = model1.predict(X_test)
print("Classification accuracy: %s" % (accuracy(y_test, y_predict)))

print ('y_test      y_predict        real val.')
for j in range(y_test.shape[0]):
    if(y_test[j] == y_predict[j]):
        print('Equal.          ', y_real[j])
    else:
        print(y_test[j], '        ', y_predict[j], '        ', y_real[j])

Classification accuracy: 0.6808510638297872
y_test      y_predict        real val.
Equal.           ['[0.06, 0.06]']
-1          1          ['[0.51, 0.51]']
Equal.           ['[10.2, 10.2]']
-1          1          ['[0.06, 0.06]']
Equal.           ['[0.08, 0.08]']
Equal.           ['[0.04, 0.04]']
Equal.           ['[2.87, 2.87]']
1          -1          ['[0.25, 0.25]']
-1          1          ['(100.0, inf)']
Equal.           ['(100.0, inf)']
Equal.           ['(100.0, inf)']
Equal.           ['(100.0, inf)']
Equal.           ['(100.0, inf)']
-1          1          ['[0.014, 0.014]']
1          -1          ['(100.0, inf)']
-1          1          ['(100.0, inf)']
Equal.           ['(100.0, inf)']
Equal.           ['(100.0, inf)']
Equal.           ['[0.12, 0.12]']
Equal.           ['[1.61, 1.61]']
Equal.           ['[1.02, 1.02]']
Equal.           ['[0.06, 0.06]']
Equal.           ['[3.23, 3.23]']
-1          1          ['[0.75, 0.75]']
Equal.           ['[0.02, 0.02]']
Equal.           